<a href="https://colab.research.google.com/github/kadirov1194/korea_energy_forecasting/blob/main/2021%EB%85%84_%EC%A0%84%EB%A0%A5%EC%88%98%EC%9A%94(%EC%86%8C%EB%B9%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 : 전력거래소 시간대별 수요량 데이터, 연료원별 거래량 데이터 활용 기간 : 2021년

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = (14, 9)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus']

In [ ]:
from google.colab import drive  # Mount to Google Colab/ MyDrive/ Colab
drive.mount('/content/drive')

In [ ]:
# (2021년) 에너지원별 누적 그래프
def energy_type_stack():
    data = pd.read_excel('/content/drive/MyDrive/국가 전력소비/한국전력거래소_연료원별 전력거래량_20220430.xlsx')
    # [행 : 거래시간, 열 : 연료원]으로 변환
    table = pd.pivot_table(data, values='전력거래량', index='거래시간', columns='연료원')

    temp_col = ['원자력', '유연탄', 'LNG', '무연탄', 'IGCC',  'LPG', '가스압', '경유', '기타', '매립가스',  '바이오가스',
     '바이오매스', '바이오중유', '부생가스', '소수력', '수력', '양수', '연료전지',   '중유', '태양광', '폐기물', '풍력', '해양에너지']
    table = table[temp_col] # 거래량 비중이 높은 에너지원별로 재정렬 (상위 : 원전, 유언탄, LNG까지만)

    table.plot(kind='area', stacked=True) # 누적 그래프
    plt.grid()
    plt.ylabel('평균 전력거래량(MWh)')
    plt.title('시간대별 에너지원별 평균 전력거래량(MWh)')
    plt.show()

In [ ]:
energy_type_stack()

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/국가 전력소비/한국전력거래소_시간별 전력수요량_20211231.xlsx', index_col=[0], parse_dates=True)
df

In [ ]:
# (2021년) 시간대별 전력피크 횟수 계산
def count_peak_month_2021(data):
    data = data['2021'] # 2021년만 선택
    data = data.idxmax(axis=1) # 하루 중 최고값을 기록한 시간을 추출
    data = data.to_frame()
    data['month'] = data.index.month
    data.columns = ['peak_hour', 'month']
    #print(data)

    total = pd.DataFrame(index=data.columns)

    for idx in data['month'].unique():
        month = data[data['month'] == idx] # 월별 데이터

        peak_hour = month['peak_hour'].value_counts() # 'peak' 컬럼에 있는 값별 count
        peak_hour = pd.DataFrame(peak_hour)

        total = pd.concat([total, peak_hour], axis=1) # 월별로 계속 결합

    total.columns = data['month'].unique() # 전체 월을 컬럼으로 변경
    total = total.fillna(0) # 피크가 기록되지 않은 시간대의 Nan 값을 0으로 변경
    total = total.T.drop(['month', 'peak_hour'], axis=1) # 행/열 전환
    total = total.stack().reset_index() # 열('시간')을 행으로 전환하고, idnex를 reset
    total.columns = ['month', 'hour', 'count'] # 컬럼명 변경

    sns.barplot(data=total, x='month', y='count', hue='hour') # 멀티 막대 그래프 그리기

    plt.grid()
    plt.ylabel('시간대별 전력피크 횟수')
    plt.title('2021년 월별/시간대별 전력피크 횟수')
    plt.show()

In [ ]:
count_peak_month_2021(df.copy())

In [ ]:
# (2021년) 부하율 그래프 그리는 함수(일반 라인, 월별 4분위)
def load_factor(data):
    data = data['2021'].copy() # 2021년 데이터만 추출
    data['LF'] = data.mean(axis=1) / data.max(axis=1) * 100 # 일별 부하율 산정

    data['LF'].plot() # 부하율 라인그래프 그리기
    plt.grid()
    plt.ylabel('부하율(%)')
    plt.title('2021년도 일별 부하율(%)')
    plt.show()

    data['month'] = data.index.month
    data4 = data[['LF', 'month']]
    #print(data4.head())

    data4.boxplot(column='LF', by='month') # 월별로 부하율을 4분위 그래프 그리기
    plt.ylabel('부하율(%)')
    plt.title('부하율(%) 4분위 그래프')
    plt.show()

In [ ]:
load_factor(df.copy())

In [ ]:
# (2021년) 연간 시간대별, 요일별, 주별 그래프 그리기
def EDA_2021(data):
    data = data['2021'].copy()  # 2021년 데이터만 추출
    stack_df = data.stack()

    stack_df = stack_df.reset_index()
    stack_df.columns = ['date', 'hour', 'pcon']
    stack_df['dayofweek'] = stack_df['date'].dt.dayofweek
    stack_df['weekofyear'] = stack_df['date'].dt.weekofyear
    #print(stack_df)

    sns.pairplot(stack_df, x_vars=['hour'], y_vars='pcon', hue='hour', height=5, plot_kws={'alpha':0.1, 'linewidth':0})
    plt.ylabel('전력소비량(MWh)')
    plt.suptitle('연간 시간대별 전력소비량(MWh) 분포')
    plt.grid()
    plt.show()

    sns.pairplot(stack_df, x_vars=['dayofweek'], y_vars='pcon', hue='hour', height=5, plot_kws={'alpha': 0.1, 'linewidth': 0})
    plt.ylabel('전력소비량(MWh)')
    plt.suptitle('연간 요일별/시간대별 전력소비량(MWh) 분포(월, 화, 수, 목, 금, 토, 일)')
    plt.grid()
    plt.show()

    sns.pairplot(stack_df, x_vars=['weekofyear'], y_vars='pcon', hue='hour', height=5, plot_kws={'alpha': 0.1, 'linewidth': 0})
    plt.ylabel('전력소비량(MWh)')
    plt.suptitle('연간 주별/시간대별 전력소비량(MWh) 분포')
    plt.grid()
    plt.show()

In [ ]:
EDA_2021(df.copy())

In [ ]:
# (2021년) 일, 월 전력소비량
def simple_EDA_2021(data):
    df_day = data['2021'].copy() # 2021년 데이터 추출
    df_day['sum'] = df_day.sum(axis=1) # 일별 전력소비량 합계 구함

    df_day['sum'].plot()
    plt.grid()
    plt.ylabel('전력소비량(MWh)')
    plt.title('2021년 일별 전력소비량(MWh)')
    plt.show()

    df_month = data['2021'].copy() # 2021년 데이터 추출
    df_month['sum'] = df_month.sum(axis=1) # 일별 전력소비량 합계 구함
    df_month['month'] = df_month.index.month
    grp_month = df_month.groupby('month')['sum'].sum()/1000 # 월별 전력소비량 합계 구함

    grp_month.plot(kind='bar')
    plt.grid()
    plt.ylabel('전력소비량(GWh)')
    plt.title('2021년 월별 전력소비량(GWh)')
    plt.show()

In [ ]:
simple_EDA_2021(df.copy())